In [ ]:
#%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import marshaltools
from ast import literal_eval
import logging
from astropy.time import Time
from ipywidgets import interactive
import ipywidgets as widgets
import re, os
import sncosmo
import json

from utils import get_config

In [ ]:
# If you are running this notebook for the first time, you will be prompted to enter your username and
# the path to your downloads from slack. If you entered something incorrectly, you can correct this in .config
config = get_config()

username = config['username']
date = '2018-09-26'
maxz = 0.1
minpeakmag = 19.5   # A candidate need to have at least one detection brigther than this
mindet = 5          # A candidate need to have at least this many detections
maxage = 30         # If a detection has an age older than this, skip (stars,age). 
                    # Q: How can histories be older than 30days?
                    # At this stage we will also cut history older than this in fit
SNcut = 5           # Need at least some photometry with this Signal to Noise

marshal_savid = 42
logpath = 'logs/'

# Get the other set of marshal source ids. Lets not talk about, took too much of my life alrady
with open('sne_following.json', 'r') as fp:
    sne_following = json.load(fp)

# log
logger = logging.getLogger(username)
handler = logging.FileHandler(os.path.join(logpath, 'ztfcosmo_trigger_%s_%s.log'%(date,username)))
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

manual_inspection=[]

In [ ]:
# Now fun starts for real. 
pl = marshaltools.ProgramList('AMPEL Test',load_candidates=False)

In [ ]:
# This is the timeconsuming part. Downloading all the data and do saltfits. Let it start and get a coffee
for count, sn in enumerate(pl.sources.keys()):
    if not pl.sources[sn]['classification'] is None:
        logger.info("SN %s classified with type %s, skipping."%(sn,pl.sources[sn]['classification']))
        continue

    if 'SNCOSMOFIT' in pl.sources[sn]:
        print('already done')
        continue
        
    # Download specific LC
    lc = pl.get_lightcurve(sn)
    del pl.lightcurves[sn]

    # Check nbr filters filters
    if len(np.unique(lc.table["filter"][lc.table["magpsf"]<99] ))<2:
        logger.info("SN %s has too few filters for saltfit."%(sn))
        continue
    
    
    # create a model
    model = sncosmo.Model(source='salt2')
    
    # Massage data
    data = lc.table_sncosmo
    data["mjd"] = data["mjd"]-Time.now().mjd
    data = data[ (data["mjd"]>-maxage) ]
    
    # Replicating data quality checks
    signalNoise = np.abs(data["flux"])/data["fluxerr"]
    data = data[ (signalNoise>SNcut) ]
    
    
    
    # run the fit
    if len(signalNoise[signalNoise>SNcut])==0:
        manual_inspection.append(sn)
        logger.info("Not enugh good data for SN %s. Manual inspection prob needed."%(sn))
        continue    
        
    try:
        result, fitted_model = sncosmo.fit_lc(
            data, model,
            ['z', 't0', 'x0', 'x1', 'c'],  # parameters of model to vary
            bounds={'z':(0., 0.2),'x1':(-5.,5.),'t0':(-20.,15.),'c':(-1.,10.)})  # bounds on parameters (if any)
        pl.sources[sn]['SNCOSMOfit'] = result, fitted_model, data
#        sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
#        plt.show()

            
    except ValueError:
        print("Why does fit fail for %s?"%(sn))
        manual_inspection.append(sn)
        logger.info("SALT fit fails for SN %s. Manual inspection prob needed."%(sn))
        continue

        
        

In [ ]:
visual_inspect = []
for sn in pl.sources.keys():
    if not 'SNCOSMOfit' in pl.sources[sn].keys():
        continue
    visual_inspect.append(sn)
print("Found %s lightcurves to inspect"%(len(visual_inspect)))     



In [ ]:
# Function to make stupid plot of a SN lightcurve
def plotty(i):
    """
    Flash SNCOSMO fit plot
    """
    
    global snnbr
    
    
    # Evaluate reply
    snname = visual_inspect[snnbr]
    if i == 'TooLate':
        logger.info("%s Faded too much while waiting. Call P200 or Keck?"%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'VeryGood':
        logger.info("Nice looking likely SN %s."%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'Possibly':
        logger.info("Get SN %s if we can."%(snname))
        decisions[snnbr] = i
        snnbr += 1    
    elif i == 'NoSNIa':
        logger.info("Object %s most likely not SNIa. "%(snname))
        decisions[snnbr] = i
        snnbr += 1
    elif i == 'BadData':
        logger.info("Too little data of %s, skip. "%(snname))
        decisions[snnbr] = i
        snnbr += 1    
    elif i == 'GoBack':
        logger.info("I want to scan some more, go back!")
        snnbr -= 1
    elif i=='Nothing':
        # Lets do nothing
        pass
    
    # Are we done?
    if snnbr==len(visual_inspect):
        print( "Seems like we are all done. How do we exit?")
        return False
    
    
    
    # Reset    
    snname = visual_inspect[snnbr]
    
    # Retrieve saltfit
    result, fitted_model, data = pl.sources[snname]['SNCOSMOfit']  
    sncosmo.plot_lc(data, model=fitted_model, errors=result.errors, 
                    figtext='%s (%i/%i)'%(snname,snnbr+1,len(visual_inspect)))
    plt.show()
    wiggy.value = 'Nothing'
    
    return (i)


wiggy = widgets.RadioButtons(
    options=['Nothing','TooLate', 'VeryGood', 'Possibly','NoSNIa','BadData','GoBack'],
    value='Nothing',
    description='Action:',
    disabled=False
)
y = interactive(plotty,i = wiggy)

In [ ]:
# Presumably you want to scan from the first SN, but you can change this and run display again to revisit something
# You cant jump ahead in the list though
snnbr = 0


In [ ]:
# This is the all important list of decisions you have made
decisions = {}

In [ ]:
# This is the scanning box! 
# You have three choices (Nothing is not a choice and GoBack steps lets you go back in order)
# - Wait : Select this if the transient is rizing and might get into RCF range (~<18.7)
# - Submit : Should probably get a spectrum (you do not need to worry about where just yet)
# - Garbage : Variable star or clearly non SNIa. Evanetually these will be rejected and never more show up
display(y)

In [ ]:
# Go through current list of SNe on the obs queue. If already 
for snnbr, choice in decisions.items():
    if not (choice=='VeryGood' or choice=='Possibly'):
        continue
    sn = visual_inspect[snnbr]
    print("%s %s"%(sn,choice))
    if sn in sne_following.keys():
        logger.info("%s already in obs queue. Adding with setting %s."%(sn,choice))
        sne_following[sn].append([choice,username,date])
    else:
        logger.info("Adding %s to obs queue with setting %s."%(sn,choice))
        sne_following[sn] = [ [choice,username,date] ]

In [ ]:
# Unfortunately not done yet. First check whether any of our SNe to obs are bright enough for RCF.
for snname in sne_following.keys():
    
    lc = pl.get_lightcurve(snname)
    del pl.lightcurves[snname]
    peakmag = np.min(lc.table['magpsf'])
    recentmag =  np.mean(lc.table['magpsf'][ (lc.table['jdobs']-Time.now().jd)>-5 ] )
    
    lastchoice = sne_following[snname][-1][0]
    
    if peakmag<18.7 and recentmag < 20:
        logger.info("%s could be RCF target? peakmag %s recentmag %s."%(sn,peakmag,recentmag))
        msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(snname)
        print( "Follow link and trigger (RCF) SEDM obs with prio %s"%(lastchoice) )
        print(msg)
    elif peakmag<19.5 and recentmag < 20:
        logger.info("%s could be Cosmology SEDM target? peakmag %s recentmag %s."%(sn,peakmag,recentmag))
        msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(snname)
        print( "Follow link and trigger (Cosmologz) SEDM obs with prio %s"%(lastchoice) )
        print(msg)
    


In [ ]:
# Now we have to go back to the sne selected for manual verification below:
# Trigger observations of these and add name below
for sn in manual_inspection:
    msg = 'http://skipper.caltech.edu:8080/cgi-bin/growth/view_source.cgi?name=%s'%(sn)
    print( "Suggested have a manual look at %s"%(sn) )
    print(msg)
    

In [ ]:
# Save the list of SN w triggered obs
with open('sne_following.json', 'w') as fp:
    json.dump(sne_following, fp)


In [ ]:
# Enter the names of the SNe you triggered SEDM obs for
sedmtrig = []
logger.info("Triggered SEDM obs of: %s"%(sedmtrig))